Get team size only during the event

In [ ]:

# ALTER TABLE projects_clean
# ADD COLUMN team_contributor_size_during integer;

# UPDATE projects_clean
# SET team_contributor_size_during =
#     COALESCE(array_length(contributors_during, 1), 0);


Extract contributors based on commit api during the event

In [ ]:

# INSERT INTO users_during (user_id)
# SELECT DISTINCT trim(u) AS user_id
# FROM projects_clean
# CROSS JOIN LATERAL unnest(contributors_during) AS u
# WHERE contributors_during IS NOT NULL
#   AND trim(u) <> '';


"""
SELECT count(*) FROM users_during;
 count
-------
 69253
"""

Remove bots

In [ ]:



# -- For users in table users, bots already removed:
# CREATE TABLE users_innerjoin AS
# SELECT u.*
# FROM users u
# INNER JOIN users_during d
#     ON u.user_id = d.user_id;

# CREATE TABLE public.users_new AS
# SELECT user_id
# FROM public.users_during
# WHERE user_id NOT ILIKE '%bot%'  
# EXCEPT
# SELECT user_id
# FROM public.users;

"""
 SELECT count(*) FROM users_innerjoin;
 count
-------
 63603

 SELECT count(*) FROM users_new;
 count
-------
  5556
"""

Remove bots

In [ ]:
# BEGIN;
# SELECT count(*) FROM public.users_during WHERE user_id ILIKE '%bot%';
# DELETE FROM public.users_during WHERE user_id ILIKE '%bot%';
# COMMIT;

Build user-project map

In [ ]:
# CREATE TABLE user_projects (
#     user_project_id SERIAL PRIMARY KEY,
#     user_id TEXT NOT NULL,
#     project_id INT NOT NULL,
#     CONSTRAINT fk_user
#         FOREIGN KEY (user_id)
#         REFERENCES users_during(user_id),
#     CONSTRAINT fk_project
#         FOREIGN KEY (project_id)
#         REFERENCES projects_clean(project_id)
# );
# ALTER TABLE user_projects
# ADD CONSTRAINT uniq_user_project UNIQUE (user_id, project_id);

# INSERT INTO user_projects (user_id, project_id)
# SELECT
#     u.user_id,
#     p.project_id
# FROM projects_clean p
# JOIN LATERAL unnest(p.contributors_during) AS c(user_id)
#     ON p.contributors_during IS NOT NULL
# JOIN users_during u
#     ON u.user_id = c.user_id
# ON CONFLICT DO NOTHING;